In [13]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.preprocessing import Normalizer


In [14]:
data=pd.read_csv('data/my.csv', index_col=None)
traindata = data.sample(frac=0.7)
testdata =data[~data.index.isin(traindata.index)]

traindata.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,25,0.17.1,0.03,0.17.2,0.24,0.25,0.26,0.05,0.27,normal
2932,0,tcp,http,SF,317,2491,0,0,0,0,...,255,1.00,0.00,0.01,0.02,0.0,0.0,0.00,0.0,normal
73581,0,tcp,private,S0,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,1.0,1.0,0.00,0.0,neptune
20957,0,tcp,private,S0,0,0,0,0,0,0,...,5,0.02,0.09,0.00,0.00,1.0,1.0,0.00,0.0,neptune
125086,0,udp,domain_u,SF,44,130,0,0,0,0,...,212,0.81,0.03,0.81,0.01,0.0,0.0,0.13,0.0,normal
56125,0,udp,domain_u,SF,44,133,0,0,0,0,...,126,0.60,0.01,0.00,0.00,0.0,0.0,0.00,0.0,normal


In [15]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in traindata.columns:
    #print df[c].dtype
    if traindata[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        traindata[c] = encodings[c].fit_transform(traindata[c])

In [16]:
encodings1 = dict()
for c in testdata.columns:
    #print df[c].dtype
    if testdata[c].dtype == "object":
        encodings1[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings1[c]
        testdata[c] = encodings1[c].fit_transform(testdata[c])

<ipython-input-16-0a51b2024455>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata[c] = encodings1[c].fit_transform(testdata[c])


In [17]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]


scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)


In [18]:
from time import time

from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3,loss='squared_hinge',alpha = 1)
t0 = time()
clf.fit(traindata,trainlabel)
tt = time() - t0
print ("Classifier trained in {} seconds.".format(round(tt, 3)))
clf

Classifier trained in 2.274 seconds.


SGDClassifier(alpha=1, loss='squared_hinge')

In [19]:
#Predictions on the test data
t0 = time()
pred = clf.predict(testdata)
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

Predicted in 0.015 seconds


In [20]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(testlabel, pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(testlabel, pred))
print('Report : ')
print(classification_report(testlabel, pred))


Confusion Matrix :
[[    0     0     0     0     0     0     0     0     0     0     0   284
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0    15
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     2
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0    12
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     3
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0    48     0   986
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     3
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0   

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
t0 = time()
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
predicted = cross_val_predict(clf, traindata,trainlabel, cv=skf)
print('Accuracy Score :',accuracy_score(trainlabel, predicted))
print('Report : ')
print(classification_report(trainlabel, predicted))
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy Score : 0.8539804944431844
Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       672
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00        41
           4       0.00      0.00      0.00         8
           5       0.00      0.00      0.00      2565
           6       0.00      0.00      0.00        15
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         5
           9       0.85      0.99      0.92     28833
          10       0.00      0.00      0.00      1073
          11       0.85      0.99      0.92     47163
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00       139
          15       0.00      0.00      0.00      2014
          16       0.00      0.00  

In [22]:
#Predictions on the test data
t0 = time()
pred = cross_val_predict(clf, testdata,testlabel, cv=skf)
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


Predicted in 7.091 seconds


In [23]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(testlabel, pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(testlabel, pred))
print('Report : ')
print(classification_report(testlabel, pred))


Confusion Matrix :
[[    0     0     0     0     0     0     0     0     0     0     0   284
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0    15
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     2
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0    12
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     3
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0    49     0   985
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     3
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0   

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
